In [264]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

class FundamentalAnalysis:
    def __init__(self, Api_Key, Url_Base, Symbol, interval, type):
        self.Url_Base = Url_Base
        self.Symbol = Symbol
        self.Api_Key = Api_Key
        self.interval = interval
        self.type = type
        self.netIncome = self.get_netIncome(Url_Base, Symbol, Api_Key)
        self.df_oil = self.get_crude_oil_prices(Url_Base, type, interval, Api_Key)
        self.df_Crude_Oil_filtered, self.netIncome_filtered = self.create_datetime(self.netIncome, self.df_oil)
        self.Crude_Oil_scaled, self.netIncome_scaled = self.scale_data(self.netIncome_filtered, self.df_Crude_Oil_filtered)
        self.plot_multiple_oil_types(self.Crude_Oil_scaled, self.netIncome_scaled)

    @staticmethod
    def get_netIncome(Url_Base, Symbol, Api_Key):
        url = f"{Url_Base}function=INCOME_STATEMENT&symbol={Symbol}&apikey={Api_Key}"
        response = requests.get(url)
        data = response.json()
        df = pd.json_normalize(data, 'quarterlyReports')
        netIncome = df.set_index('fiscalDateEnding')['netIncome'].astype(float)
        return netIncome

    @staticmethod
    def get_crude_oil_prices(Url_Base, type, interval, Api_Key):
        url = f"{Url_Base}function={type}&interval={interval}&apikey={Api_Key}"
        response = requests.get(url)
        data = response.json()
        df_Crude_Oil = pd.DataFrame(data['data'], columns=['date', 'value'])
        df_Crude_Oil.set_index('date', inplace=True)
        df_Crude_Oil['value'] = pd.to_numeric(df_Crude_Oil['value'], errors='coerce')
        return df_Crude_Oil

    @staticmethod
    def create_datetime(netIncome, df_Crude_Oil):
        df_Crude_Oil.index = pd.to_datetime(df_Crude_Oil.index)
        netIncome.index = pd.to_datetime(netIncome.index)
        start_date = max(netIncome.index.min(), df_Crude_Oil.index.min())
        end_date = min(netIncome.index.max(), df_Crude_Oil.index.max())
        return df_Crude_Oil.loc[start_date:end_date], netIncome.loc[start_date:end_date]

    @staticmethod
    def scale_data(netIncome_filtered, df_Crude_Oil_filtered):
        scaler_oil_filtered = MinMaxScaler()
        scaler_net_income = MinMaxScaler()
        Crude_Oil_scaled = scaler_oil_filtered.fit_transform(df_Crude_Oil_filtered[['value']])
        netIncome_scaled = scaler_net_income.fit_transform(netIncome_filtered.to_frame())
        return Crude_Oil_scaled, netIncome_scaled

    @staticmethod
    def plot_multiple_oil_types(Crude_Oil_scaled, netIncome_scaled):
        plt.figure(figsize=(12, 6))
        dates = pd.date_range(start='2023-01-01', periods=len(Crude_Oil_scaled), freq='W')  # Example date range
        plt.plot(dates, Crude_Oil_scaled, label='WTI Oil Price (Scaled)')
        plt.plot(dates, netIncome_scaled, label='Company Net Income (Scaled)', color='orange', linestyle='--')
        plt.xlabel


In [263]:


Url_Base = "https://www.alphavantage.co/query?"
Symbol= "XOM" #this is the ticker symbol for intel
interval = "weekly"
type = "WTI"
Api_Key = 'OFTFU0NSZ98V2ERT'

run_program = Fundemental_Analysis(type, Url_Base, Symbol, interval, Api_Key)


TypeError: Markdown expects text, not {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}